## Mobile App for Lottery Addiction

### Introduction

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the [6/49 lottery](https://en.wikipedia.org/wiki/Lotto_6/49) and build functions that enable users to answer questions like:
 - What is the probability of winning the big prize with a single ticket?
 - What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
 - What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

The institute also wants us to consider historical data coming from the national 6/49 lottery game in Canada. [The data set](https://www.kaggle.com/datasets/datascienceai/lottery-dataset) has data for 3,665 drawings, dating from 1982 to 2018.


## Core Functions

we'll need to calculate repeatedly probabilities and combinations. As a consequence, we'll start by writing two functions that we'll use often:

 - A `function()` that calculates factorials; and
 - A `function()` that calculates combinations.


In [1]:
def factorial(n):
    final_product = 1
    for i in range(n, 0, -1):
        final_product *= i
    return final_product

def combinations(n, k):
    numerator = factorial(n)
    denominator = factorial(k) * factorial(n-k)
    return numerator/denominator


### One-ticket Probability

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. A player wins the big prize if the six numbers on their tickets match all the six numbers drawn.

For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49).

In [2]:
def one_ticket_probability(user_number):
    n_combination = combinations(49, 6)
    probability_one_ticket = 1 / n_combination
    percentage_form = probability_one_ticket * 100
    
    print('''Your change of win the big prize with the number {} are {:.7f}%.
    In other words, you have 1 in {:,} chances to win'''.format(user_number,
                                                               percentage_form, int(n_combination)))
    

Now, we can test our functions on different input.

In [3]:
test_input =[13, 22, 24, 27, 42, 44] 
one_ticket_probability(test_input)

Your change of win the big prize with the number [13, 22, 24, 27, 42, 44] are 0.0000072%.
    In other words, you have 1 in 13,983,816 chances to win


In [4]:
test_input = [2, 4, 35, 24, 10, 15]
one_ticket_probability(test_input)

Your change of win the big prize with the number [2, 4, 35, 24, 10, 15] are 0.0000072%.
    In other words, you have 1 in 13,983,816 chances to win


### Historical Data Check for Canada Lottery

We wrote a function that can tell users what is the probability of winning the big prize with a single ticket.

We'll focus on exploring the historical data coming from the Canada 6/49 lottery. The data set can be download from [Kaggle](https://www.kaggle.com/datasets/datascienceai/lottery-dataset).

The data set contains historical data for 3,665 drawings (each row shows data for a single drawing), dating from 1982 to 2018.  
 

In [5]:
import pandas as pd 

lottery = pd.read_csv('649.csv')
lottery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3665 entries, 0 to 3664
Data columns (total 11 columns):
PRODUCT            3665 non-null int64
DRAW NUMBER        3665 non-null int64
SEQUENCE NUMBER    3665 non-null int64
DRAW DATE          3665 non-null object
NUMBER DRAWN 1     3665 non-null int64
NUMBER DRAWN 2     3665 non-null int64
NUMBER DRAWN 3     3665 non-null int64
NUMBER DRAWN 4     3665 non-null int64
NUMBER DRAWN 5     3665 non-null int64
NUMBER DRAWN 6     3665 non-null int64
BONUS NUMBER       3665 non-null int64
dtypes: int64(10), object(1)
memory usage: 315.0+ KB


In [6]:
#see the firts three rows
lottery.head(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34


In [7]:
#see the last 3 rows
lottery.tail(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


### Function for historical Data Check

we're going to write `extract_numbers()` a function that will enable users to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now. 

The engineering team told us that we need to be aware of the following details:
 - Inside the app, the user inputs six different numbers from 1 to 49.
 - Under the hood, the six numbers will come as a Python list and serve as an input to our function.
 - The engineering team wants us to write a function that prints:
     - the number of times the combination selected occurred in the Canada data set; and
     - the probability of winning the big prize in the next drawing with that combination.

In [8]:
def extract_numbers(row):
    row = row[4:10]
    row = set(row.values)
    return row

winning_numbers = lottery.apply(extract_numbers, axis=1)
winning_numbers.head()

0    {3, 41, 11, 12, 43, 14}
1    {33, 36, 37, 39, 8, 41}
2     {1, 6, 39, 23, 24, 27}
3     {3, 9, 10, 43, 13, 20}
4    {34, 5, 14, 47, 21, 31}
dtype: object

Now, we'll write a function `check_istorical_occurence()` that take in the input user and the storical data and print information with respect to the number of occurrence and the probability of winning in the next  drawing. 

In [9]:
def check_historical_occurrence(user_numbers, historical_numbers):
    '''user number: Python list
        Historical number: panda series'''
    user_numbers_set = set(user_numbers)
    check_occurrences = historical_numbers == user_numbers_set
    n_occurrences = check_occurrences.sum()
    
    if n_occurrences == 0:
        print('''The combinations {} never occurred.
 it doesn't mean it's more likely to occur now. Your chances to win
the big prize in the next drawing, 
In other words, you have 1 in 13,983,816 chances to win'''.format(user_numbers, user_numbers))
        
    else: 
        print('''The number of time combinations {} has occured in the past is {}.
 Your change of win the big prize with the number {} are 0.0000072%.
In other words, you have 1 in 13,983,816 chances to win.'''.format(user_numbers, n_occurrences,
                                                                                user_numbers))

In [10]:
test_input_2 = [33, 36, 37, 39, 8, 41]
check_historical_occurrence(test_input_2, winning_numbers)

The number of time combinations [33, 36, 37, 39, 8, 41] has occured in the past is 1.
 Your change of win the big prize with the number [33, 36, 37, 39, 8, 41] are 0.0000072%.
In other words, you have 1 in 13,983,816 chances to win.


In [11]:
test_input_3 = [3, 2, 44, 22, 1, 44]
check_historical_occurrence(test_input_3, winning_numbers)

The combinations [3, 2, 44, 22, 1, 44] never occurred.
 it doesn't mean it's more likely to occur now. Your chances to win
the big prize in the next drawing, 
In other words, you have 1 in 13,983,816 chances to win


### Multi-ticket Probability

Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. 

Our purpose is to help them better estimate their chances of winning.

We've talked with the engineering team and they gave us the following information.

 - The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
 - Our function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
 - The function shoul print information about the provability of winning the big prize depending on the number of different tickets played.
 
 below, we write the function `multi_ticket_probability()` that take in
 the number of ticketrs, and print probability information depending on the input.

In [12]:
def multi_ticket_probability(n_tickets):
    n_combinations = combinations(49, 6)
    
    probability_ticket = n_tickets/n_combinations
    percentage_form = probability_ticket * 100
    
    if n_tickets == 1:
        print('''Your change to win the big prize with one ticket are {:.6f}%.
in other words, the chaces to win.'''.format(percentage_form, int(n_combinations)))
              
    else:
        combinations_simplified = round(n_combinations/n_tickets)
        print('''The chance to win the big prize with {:,} different ticket are {:.6f}%.
in other words you have a 1 in {:,} to win'''.format(n_tickets, percentage_form,
                                                     combinations_simplified))       


In [13]:
test_inputs = [1, 10, 100, 10000, 1000000, 6991908, 13983816]
for test_input in test_inputs:
    multi_ticket_probability(test_input)
    print('-----------------------')

Your change to win the big prize with one ticket are 0.000007%.
in other words, the chaces to win.
-----------------------
The chance to win the big prize with 10 different ticket are 0.000072%.
in other words you have a 1 in 1,398,382 to win
-----------------------
The chance to win the big prize with 100 different ticket are 0.000715%.
in other words you have a 1 in 139,838 to win
-----------------------
The chance to win the big prize with 10,000 different ticket are 0.071511%.
in other words you have a 1 in 1,398 to win
-----------------------
The chance to win the big prize with 1,000,000 different ticket are 7.151124%.
in other words you have a 1 in 14 to win
-----------------------
The chance to win the big prize with 6,991,908 different ticket are 50.000000%.
in other words you have a 1 in 2 to win
-----------------------
The chance to win the big prize with 13,983,816 different ticket are 100.000000%.
in other words you have a 1 in 1 to win
-----------------------


### Less Winning Numbers - Function

we're going to write one more function to allow the users to calculate probabilities for two, three, four, or five winning numbers.

 in most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

These are the engineering details we'll need to be aware of:

- Inside the app, the user inputs:
  - six different numbers from 1 to 49; and
  - an integer between 2 and 5 that represents the number of winning numbers expected
- Our function prints information about the probability of having the inputted number of winning numbers.

To calculate the probabilities, we tell the engineering team that the specific combination on the ticket is irrelevant behind the scenes, and we only need the integer between 2 and 5 representing the number of winning numbers expected. We will write a function named `probability_less_6()` which takes in an integer and prints information about the chances of winning depending on the value of that integer.


In [18]:
def probability_less_6(n_winning_numbers):
    n_combinations_ticket = combinations(6, n_winning_numbers)
    n_combinations_rimaining = combinations(43, 6 - n_winning_numbers)
    succeful_outcomes = n_combinations_ticket * n_combinations_rimaining
    
    n_combination_total = combinations(49, 6)
    probability = succeful_outcomes/n_combination_total
    
    probability_percentage = probability * 100
    
    combinations_simplified = round(n_combination_total/succeful_outcomes)
    print('''Your changes of having {} winning number with this ticket are {:.6f}%.
in other words, you have a 1 in {} chances to win'''.format(n_winning_numbers, probability_percentage,
                                                           int(combinations_simplified)))

In [19]:
 for test_output in [1, 2, 3, 4, 5]:
        probability_less_6(test_output)
        print('------------------------')

Your changes of having 1 winning number with this ticket are 41.301945%.
in other words, you have a 1 in 2 chances to win
------------------------
Your changes of having 2 winning number with this ticket are 13.237803%.
in other words, you have a 1 in 8 chances to win
------------------------
Your changes of having 3 winning number with this ticket are 1.765040%.
in other words, you have a 1 in 57 chances to win
------------------------
Your changes of having 4 winning number with this ticket are 0.096862%.
in other words, you have a 1 in 1032 chances to win
------------------------
Your changes of having 5 winning number with this ticket are 0.001845%.
in other words, you have a 1 in 54201 chances to win
------------------------


### Next Stepds

 We managed to write four main functions for our app:
 - one_ticket_probability() — calculates the probability of winning the big prize with a single ticket
 - check_historical_occurrence() — checks whether a certain combination has occurred in the Canada lottery data set
 - multi_ticket_probability() — calculates the probability for any number of of tickets between 1 and 13,983,816
 - probability_less_6() — calculates the probability of having two, three, four or five winning numbers
 
 Possible features for a second version of the app include:

 - Making the outputs even easier to understand by adding fun analogies (for example, we can find probabilities for strange events and compare with the chances of winning in lottery; for instance, we can output something along the lines "You are 100 times more likely to be the victim of a shark attack than winning the lottery").
 - Combining the one_ticket_probability() and check_historical_occurrence() to output information on probability and historical occurrence at the same time.
 - Creating a function similar to probability_less_6() which calculates the probability of having at least two, three, four or five winning numbers. Hint: the number of successful outcomes for having at least four winning numbers is the sum of these three numbers:
The number of successful outcomes for having four winning numbers exactly
The number of successful outcomes for having five winning numbers exactly
The number of successful outcomes for having six winning numbers exactly
 